## Workflow for Design of Experiment: _More General_

In [ ]:
import numpy as np
import pandas as pd
from pyDOE2 import *
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from IPython.display import display, HTML
import plotly.graph_objs as go
import os

import data_processing as dp

#### This cell will contain a function to select among different design options like Full Factorial, Fractional Factorial, etc., and create the design table.

* The function _create_design_ is designed to accommodate different experimental needs by creating several types of designs like Full Factorial, Fractional Factorial, Plackett Burman, Box Behnken, Central Composite, and Latin Hypercube designs. 

#### Design Types:

* General Full Factorial: Generates a full factorial design with a specified number of levels and factors.
* 2 Level Full Factorial: Creates a 2-level full factorial design for a specified number of factors.
* 2 Level Fractional Factorial: Produces a fractional factorial design based on a generator string that indicates the allowed confounding.
* Plackett Burman: Used for generating fractional-factorial designs, especially for a large number of factors.
* Box Behnken: Suitable for 3-level designs and includes a central point for each factor. Often used for response surface methodology.
* Central Composite: Generates a central composite design, commonly used in response surface methodology.
* Latin Hypercube: Creates a Latin hypercube sample design, useful for creating random samples of parameter values from a multidimensional distribution.

#### Additional Notes

* It's important to refer to the documentation of pyDOE2 and the NIST handbook provided in the function comments for detailed explanations of each design type.
* _Design Selection:_ Choose the design type based on your experimental needs. For example, full factorial designs are comprehensive but may require a large number of experiments, while fractional factorial designs like Plackett Burman and Box Behnken can be more efficient.

In [ ]:
def create_design(design_type, *args):
    '''
    Refer to:
     # https://pythonhosted.org/pyDOE/
     # https://www.itl.nist.gov/div898/handbook/pri/section3/pri33.htm
     # https://github.com/clicumu/pyDOE2/tree/master
    
    # Mostly Used: 
     # 2 level full factorial design for 2 Level and # factors
     # Plackett Burman for fractional-factorial designs 
     # Box Behnken for 3 level (includes a central point) for each # factors
    
    '''
    if design_type == "General Full Factorial":
        design = fullfact(*args)       # define the number of 3 Levels for n Factors.
        #design_table = fullfact([3,3,3]) # for 3 levels in 3 Factors
        
    elif design_type == "2 Level Full Factorial":
        design = ff2n(*args)       # define the number of Factors ex. 3 for Temp, Slope, pH
        #design_table = create_design("2_level_full_factorial",3)
        
    elif design_type == "2 Level Fractional Factorial":
        design = fracfact(*args)   # the input to fracfact is a generator string of symbolic characters to indicate the 'confounding' that will be allowed
        #design_table = create_design("2_level_fractional_factorial",'a b ab')
        
    elif design_type == "Plackett Burman":
        design = pbdesign(*args)   # Similar to  generate fractional-factorial designs, define the number of Factors 
        #design_table = create_design("plackett_burman",3)
        
    elif design_type == "Box Behnken":
        design = bbdesign(*args)   # "These designs are rotatable (or near rotatable) and require 3 levels of each factor."
        #design_table = create_design("box_behnken",3)
        
    elif design_type == "Central Composite":
        design = ccdesign(*args)
        #design_table = ccdesign(3)
        
    elif design_type == "latin_hypercube":
        design = lhs(*args)
        
    else:
        raise ValueError("Design type not recognized")
    return pd.DataFrame(design)


### Check the chosen Design

* The plot visually demonstrates how the experiments are spread across the factor space, giving you an idea of the coverage and distribution of the experiments.

In [ ]:
design_use = "General Full Factorial"

levels = [3,3,3]
factors = len(levels)

# Name the variables
Factor_1_ = "%B initial"
Factor_2_ = "slope %B"
Factor_3_ = "Temperature"

design_table = create_design(design_use,levels)
design_table.rename(columns={0:Factor_1_,1:Factor_2_,2:Factor_3_},inplace=True)

# Create the 3D scatter plot
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(121, projection='3d')  # Changed to 121 to accommodate two plots side by side
ax.scatter(design_table[Factor_1_], design_table[Factor_2_], design_table[Factor_3_])

# Set labels and title
ax.set_xlabel(Factor_1_); ax.set_ylabel(Factor_2_); ax.set_zlabel(Factor_3_)
ax.set_title(f"{design_use} Design Points")

# Display the DataFrame next to the plot
plt.subplot(122)  # Position for the table
plt.axis('off')  # Turn off axis
plt.table(cellText=design_table.values, colLabels=design_table.columns, cellLoc = 'center', loc='center')
plt.show()

# Alternatively, if you just want to display it as a DataFrame and not as a table in the plot
#display(design_table)

In [ ]:
# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=design_table[Factor_1_], y=design_table[Factor_2_], z=design_table[Factor_3_], mode='markers',
    marker=dict(size=5, opacity=0.8,),
    text=[f'{Factor_1_}: {x}, {Factor_2_}: {y}, {Factor_3_}: {z}' for x, y, z in zip(design_table[Factor_1_], design_table[Factor_2_], design_table[Factor_3_])]
)])

# Update layout
fig.update_layout(
    title=f"{design_use} Design Points",
    scene = dict(
        xaxis_title=Factor_1_, # %B initial
        yaxis_title=Factor_2_, # slope %B
        zaxis_title=Factor_3_) # Temperature
)

fig.show()

# Save the figure as an HTML file in the 'images' subdirectory
os.makedirs('images', exist_ok=True)
fig.write_html('images/Model_start.html')

#design_table.rename(columns={0:Factor_1, 1:Factor_2, 2:Factor_3}, inplace=True)
#design_table

#### Setting up your experimental table

* Defining Factor Values

* Create the Experimental table that will be used in the wet- laboratory

    * The DataFrame is saved as a CSV file named "Experimental Table from DoE - screening 3F 2L.csv". The sep=";" specifies that the separator used in the file is a semicolon. index=False ensures that the DataFrame index is not included in the CSV file.



In [ ]:
Factor_1 = [5, 15, 25]  # Values for Factor 1 - %B initial
Factor_2 = ["2.5%B/min", "5%B/min", "10%B/min"]  # Values for Factor 2 - %B slope (%B/min.): (1)2.5%B/min. (2)5%B/min.; (3)10%B/min.
#Factor_2 = [f"2.5%B/min({(100-Factor_1[0]/100)/2.5:.01f} min.)", f"5%B/min({(100-Factor_1[1]/100)/5:.01f} min.)", f"10%B/min({(100-Factor_1[-1]/100)/10:.01f} min.)"]  # Values for Factor 2 - %B slope (%B/min.): (1)2.5%B/min. (2)5%B/min.; (3)10%B/min.
Factor_3 = [25, 42.5, 60]  # Values for Factor 3 - Temperature

# Mapping for each factor
mapping_factor_1 = {0: Factor_1[0], 1: Factor_1[1], 2: Factor_1[-1]}
mapping_factor_2 = {0: Factor_2[0], 1: Factor_2[1], 2: Factor_2[-1]}
mapping_factor_3 = {0: Factor_3[0], 1: Factor_3[1], 2: Factor_3[-1]}

# Assuming 'design_table' is the DataFrame with -1, 0, and +1 (ex. box_behnken design)
exp_table = design_table.copy()

# Map the factors
exp_table.iloc[:, 0] = exp_table.iloc[:, 0].map(mapping_factor_1)
exp_table.iloc[:, 1] = exp_table.iloc[:, 1].map(mapping_factor_2)
exp_table.iloc[:, 2] = exp_table.iloc[:, 2].map(mapping_factor_3)

# Add experiment number and results columns
exp_table["Experiment#"] = exp_table.index +1
exp_table['Number of Peaks'] = exp_table.apply(lambda _: '', axis=1) # Number of Peaks
exp_table['Run Time'] = exp_table.apply(lambda _: '', axis=1) # Run Time
exp_table['Results'] = exp_table.apply(lambda _: '', axis=1) # Run Time

# Rename the columns to match the factors
exp_table.rename(columns={exp_table.columns[0]: "%B initial", exp_table.columns[1]: "%B slope", exp_table.columns[2]: "Temperature"}, inplace=True)

# Reorder columns for display
exp_table = exp_table[['Experiment#', exp_table.columns[0], exp_table.columns[1], exp_table.columns[2], 'Results','Number of Peaks','Run Time']]

# Save the experimental table to a CSV file
exp_table.to_csv(f"Experimental Table from DoE - {design_use}.csv", sep=";", index=False)
print(f"Your experimental table ({design_use}) is ready as a CSV file for use.")
display(exp_table)

###### Create the pdf file with the Suggested Series of Methods for the Experiment

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.units import inch
import pandas as pd

# Create a PDF document
pdf_file = f"Suggested_Series_of_Methods_{design_use}.pdf"
document = SimpleDocTemplate(pdf_file, pagesize=A4)

# Create a list to hold the elements to be added to the PDF
elements = []

# Title
styles = getSampleStyleSheet()
title_style = styles['Title']
title = Paragraph("Suggested Series of Methods for the Experiment", title_style)
elements.append(title)

# Add some space
elements.append(Spacer(1, 12))

# Add introductory text
intro_text = Paragraph(f"""Below is the suggested series of methods to be used in the experiment,
                           using the {design_use} design with {factors} factors and {levels} levels (for each factor)
                           based on the design of experiments.
                           """, styles['BodyText'])
elements.append(intro_text)

# Add some space
elements.append(Spacer(1, 12))

# Prepare the table data
table_data = [exp_table.columns.tolist()] + exp_table.values.tolist()

# Create a table with the data
table = Table(table_data, hAlign='CENTER')

# Style the table
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
]))

# Add the table to the elements
elements.append(table)

# Add some space after the table
elements.append(Spacer(1, 24))

# Add a final comment1
comment_text = Paragraph("OBS: (%B initial. - 100)/(%B slope) = time of gradient ")
elements.append(comment_text)

# Add a final comment2
comment_text = Paragraph(f"ex.: For when %B initial is 5 and %B slope is 2.5, the gradient will reach 100%B in {(100 - Factor_1[0]) / 2.5:.2f} min", getSampleStyleSheet()['Normal'])
elements.append(comment_text)

# Add a final comment3
comment_text = Paragraph("Produced by DOEpipeline. Please cite: 'available soon'", styles['Italic'])
elements.append(comment_text)

# Build the PDF
document.build(elements)

print(f"PDF {pdf_file} created successfully.")

In [ ]:
# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=exp_table[exp_table.columns[0]], y=exp_table[exp_table.columns[1]], z=exp_table[exp_table.columns[2]], mode='markers',
    marker=dict(size=5, opacity=0.8,),
    text=[f'{exp_table.columns[0]}: {x}, {exp_table.columns[1]}: {y}, {exp_table.columns[2]}: {z}' for x, y, z in zip(exp_table[exp_table.columns[0]], exp_table[exp_table.columns[1]], exp_table[exp_table.columns[2]])]
)])

# Update layout
fig.update_layout(
    title=f"{exp_table} Design Points",
    scene = dict(
        xaxis_title=exp_table.columns[0],
        yaxis_title=exp_table.columns[1],
        zaxis_title=exp_table.columns[2])
)

# Save the figure as an HTML file in the 'images' subdirectory
os.makedirs('images', exist_ok=True)
fig.write_html('images/Model.html')

fig.show()

### Add columns that represent interaction between Factors and the results from the experiments

* This adds a new column Results to design_table, containing the results from the experiments.

#### Read Chromatograms

In [ ]:
directory_path = r'C:\Users\borge\Downloads\Cromatograma JULIANA\DoE'
os.chdir(directory_path)
%pwd

# Input/output directories and retention time range
input_folder = directory_path
output_folder = os.path.join(input_folder, 'data')
retention_time_start = 1
retention_time_end = 54

# Run the process
combined_df2 = dp.combine_and_trim_data(input_folder, output_folder, retention_time_start, retention_time_end)
combined_df2

import os
import plotly.graph_objects as go

# Your existing code to create the Plotly figure
fig = go.Figure()
start_column = 1
end_column = 53

# Define a larger gap value to separate the traces more clearly
gap = 200000  # Increased gap value for a more pronounced separation between traces

# Ensure all columns being plotted are numeric
for column in combined_df2.columns[start_column:end_column + 1]:
    combined_df2[column] = pd.to_numeric(combined_df2[column], errors='coerce').fillna(0)

# Iterate over the specified columns and add each trace with a vertical offset
for i, column in enumerate(combined_df2.columns[start_column:end_column + 1]):
    fig.add_trace(go.Scatter(
        x=combined_df2['RT(min)'],
        y=combined_df2[column] + i * gap,  # Offset each trace by 'i * gap'
        mode='lines',
        name=column
    ))

# Update the layout of the figure
fig.update_layout(
    title='Stacked Chromatograms with Larger Gaps',
    xaxis_title='RT (min)',
    yaxis_title='Intensity (Stacked)',
    legend_title='Samples',
    hovermode='closest',
    width=1000,  # Set the width of the figure
    height=1000  # Set the height of the figure
)

# Save the figure as an HTML file in the 'images' subdirectory
os.makedirs('images', exist_ok=True)
fig.write_html('images/stacked_chromatograms_larger_gap.html')

# Show the figure (optional)
fig.show()


In [ ]:
# Example: Define a list of experiments to compare
selected_experiments = [
    "DoE11",  # Replace with actual column names for the samples/experiments
    "DoE14",
    "DoE17"
]

# Ensure the selected experiments exist in the DataFrame
available_experiments = [col for col in selected_experiments if col in combined_df2.columns]

# Create a Plotly figure
fig = go.Figure()
gap = 200000  # Set the gap for separating traces

# Iterate over the selected experiments and plot
for i, column in enumerate(available_experiments):
    fig.add_trace(go.Scatter(
        x=combined_df2['RT(min)'],  # Ensure RT(min) is the time column
        y=combined_df2[column] + i * gap,  # Offset by i * gap
        mode='lines',
        name=column
    ))

# Update the layout
fig.update_layout(
    title='Selected Chromatograms Comparison',
    xaxis_title='RT (min)',
    yaxis_title='Intensity (Stacked)',
    legend_title='Samples',
    hovermode='closest',
    width=1000,
    height=1000
)

# Save the figure as an HTML file in the 'images' subdirectory
os.makedirs('images', exist_ok=True)
fig.write_html('images/selected_chromatograms_comparison.html')

# Show the figure
fig.show()

In [ ]:
from scipy.signal import find_peaks, savgol_filter
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Initialize a dictionary to store the number of peaks for each sample
peaks_count = {}

# Iterate over each sample column to detect peaks
for column in combined_df2.columns[start_column:end_column + 1]:
    # Extract the intensity values for the current sample
    intensity_values = combined_df2[column].fillna(0).values  # Replace NaNs with 0
    
    # Apply baseline correction
    baseline = savgol_filter(intensity_values, window_length=51, polyorder=3)
    corrected_intensity = intensity_values - baseline

    # Detect peaks in the corrected intensity values
    peaks, properties = find_peaks(
        corrected_intensity,
        height=1000,  # Reduced height for smaller peaks
        prominence=3000,  # Lower prominence to include less distinct peaks
        distance=2,  # Allow closely spaced peaks
        width=(2, None)  # Include broader peaks
    )

    # Count the number of detected peaks
    peaks_count[column] = len(peaks)

    # Create an interactive plot for one column
    if column == combined_df2.columns[start_column+1]:  # Plot only for the first column as an example
        fig = go.Figure()

        # Add original intensity trace
        fig.add_trace(go.Scatter(
            x=combined_df2['RT(min)'],
            y=intensity_values,
            mode='lines',
            name='Original Intensity'
        ))

        # Add corrected intensity trace
        fig.add_trace(go.Scatter(
            x=combined_df2['RT(min)'],
            y=corrected_intensity,
            mode='lines',
            name='Corrected Intensity'
        ))

        # Add baseline trace
        fig.add_trace(go.Scatter(
            x=combined_df2['RT(min)'],
            y=baseline,
            mode='lines',
            name='Baseline',
            line=dict(dash='dash')
        ))

        # Add detected peaks
        fig.add_trace(go.Scatter(
            x=combined_df2['RT(min)'][peaks],
            y=corrected_intensity[peaks],
            mode='markers',
            name='Peaks',
            marker=dict(color='red', size=10)
        ))

        # Update layout for zoomable and interactive features
        fig.update_layout(
            title=f"Interactive Peaks for {column}",
            xaxis_title="RT (min)",
            yaxis_title="Intensity",
            legend_title="Legend",
            hovermode="closest"
        )

        # Show the plot
        fig.show()

# Convert the peaks count dictionary to a DataFrame for better visualization
peaks_count_df = pd.DataFrame(list(peaks_count.items()), columns=['Sample', 'Number of Peaks'])

# Save the DataFrame as a CSV file
peaks_count_df.to_csv('images/peaks_count_with_slope.csv', index=False)


In [ ]:
for col in combined_df2.columns[start_column:end_column + 1]:
    dp.plot_interactive_peaks(combined_df2, col)

In [ ]:
B0 = np.ones(len(design_table))
design_table["B0"] = B0

# Create new columns by multiplying existing columns
design_table['Factor_1_2'] = design_table[Factor_1_] * design_table[Factor_2_]
design_table['Factor_1_1'] = design_table[Factor_1_] * design_table[Factor_1_]
design_table['Factor_2_2'] = design_table[Factor_2_] * design_table[Factor_2_]

# Load the Results into the Design Table
#results = pd.read_csv("results.csv")
Number_of_peaks = peaks_count_df["Number of Peaks"] #[28,24,23,29,28,25,20,24,24,27,22,22,26,24,22,30,27,27,25,26,23,26,27,23,29,27,29]

Run_time = [38, 34, 30, 19, 17, 15, 9.5, 8.5, 7.5, 38, 34, 30, 19, 17, 15, 9.5, 8.5, 7.5, 38, 34, 30, 19, 17, 15, 9.5, 8.5, 7.5]

#Run_time = [43,39,35,25,23,21,17,16,15,42,38,34,25,23,21,17,15,14,46,37,33,24,22,20,15,24,13]

# Create a DataFrame from the provided lists
data = {'Number_of_peaks': Number_of_peaks, 'Run_time': Run_time}

# Calculate the Results as the division of Number_of_peaks by Run_time
results = [(peaks*1.5) / (np.log(time))  for peaks, time in zip(Number_of_peaks, Run_time)]
#results = [(peaks*2) / ((time))  for peaks, time in zip(Number_of_peaks, Run_time)]
#results = Number_of_peaks

# Create the design_table DataFrame
#design_table = pd.DataFrame(data)
design_table['Results'] = results

#num_rows = len(design_table)
#random_floats = np.random.random(num_rows)
#results = pd.DataFrame({'Result': random_floats})

design_table2 = design_table[['B0', Factor_1_,# %B initial
                             Factor_2_,# slope %B
                             Factor_3_,# Temperature
                             'Factor_1_2', 'Factor_1_1','Factor_2_2', 'Results']]

design_table2

#### Effect
A linear regression analysis to model how the independent variables (Factor_1, Factor_2, their interactions, and squared terms) affect the dependent variable (Results). 
* The calculated coefficients describe the relationship, and the standard errors provide insight into the reliability of these coefficients.

In [ ]:
errorr

In [ ]:
# Extracting y-values (dependent variable) and x-values (independent variables)
y = design_table[['Results']].values
X = design_table[[Factor_1_, Factor_2_, Factor_3_, "Factor_1_2", "Factor_1_1", "Factor_2_2"]].values

# Add a column of ones to X to include an intercept in the model
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Perform linear regression
coefficients, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)

# Calculate variance of residuals
sigma_squared = residuals / (len(y) - X.shape[1])  # Adjusted for degrees of freedom

# Define factor names
factor_names = [
    "Intercept",
    Factor_1_,
    Factor_2_,
    Factor_3_,
    "Interaction (%B initial * slope %B)",
    "Squared (%B initial)^2",
    "Squared (slope %B)^2"
]

# Residual variance (sigma^2)
residual_variance = residuals / (len(y) - X.shape[1])

# Covariance matrix of coefficients
cov_matrix = residual_variance * np.linalg.inv(X.T @ X)

# Standard errors for coefficients
standard_errors = np.sqrt(np.diag(cov_matrix))

# Organize coefficients and standard errors into a DataFrame
coeff_df = pd.DataFrame({
    "Factor": factor_names,
    "Coefficient": coefficients.flatten(),
    "Standard Error": standard_errors.flatten()
})

# Display the organized output
display(coeff_df)



Interpretation of the Table:

    Intercept:
        The intercept (30.9330.93) represents the predicted response when all predictors are set to zero.
        The standard error for the intercept is relatively small, indicating good confidence in its estimate.

    Main Effects:
        %B initial and slope %B have similar standard errors (1.37 and 1.37), suggesting similar confidence in their coefficients.
        Temperature has a much smaller standard error (0.36 and 0.36), indicating higher precision for this factor's coefficient estimate.

    Interaction and Quadratic Terms:
        The interaction term and squared terms also have reasonable standard errors, reflecting the combined effects of predictors.
        The standard error of 0.62 and 0.62 for the squared terms indicates moderate variability in these coefficients.
        
Magnitude of Effect

    The absolute value of the coefficients indicates the strength of each factor’s effect on the response variable.
    Larger coefficients (positive or negative) correspond to stronger effects.

From the table:

    %B initial: Coefficient = −2.18−2.18 (strong negative effect)
    Temperature: Coefficient = 1.811.81 (strong positive effect)
    Other coefficients are smaller in magnitude, indicating weaker interactions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Generate predictions based on the model coefficients
y_pred = X @ coefficients

# Calculate residuals
residuals = y - y_pred

# Calculate R-squared value
r2 = r2_score(y, y_pred)

# Plot residuals vs fitted values
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.7, edgecolor='k')
plt.axhline(y=0, color='r', linestyle='--', linewidth=1)
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.grid(alpha=0.3)
plt.show()

# Histogram of residuals
plt.figure(figsize=(8, 6))
plt.hist(residuals, bins=15, color='skyblue', edgecolor='k', alpha=0.7)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)
plt.show()

# Display R-squared value
r2


Observations:

    Residuals vs. Fitted Values:
        The residuals appear randomly scattered around the horizontal axis (y=0), which is a good indicator of no apparent pattern in the residuals.
        This randomness suggests that the model fits the data reasonably well and that there are no obvious violations of linear regression assumptions (e.g., no strong heteroscedasticity or non-linearity).

    Histogram of Residuals:
        The residuals appear to be fairly symmetric around zero.
        While it suggests approximate normality, this would need to be formally tested with a Shapiro-Wilk test or a Q-Q plot for further confirmation.

___
### Creating the Response Surface matrix

* Creating a Response Surface matrix involves setting up a framework where you can later input the responses (or outcomes) of your experiments based on varying levels of two factors. The code you've provided sets up such a matrix with values ranging from -1 to 1 (representing the factor levels) and divides this range into 11 steps.

* When you run Resp_surf_ffact, it will display an 11x11 grid. Initially, all cells are filled with zeros. This matrix serves as a template where you can input the results of your experiments. Each cell corresponds to a combination of factor levels (one level from the horizontal axis and one from the vertical axis).

In [ ]:
vector = np.linspace(-1, 1, 11)

# Create an 11x11 DataFrame with zeros (or any default value)
Resp_surf = pd.DataFrame(np.zeros((12, 12)))

# Set the headers and index with the same vector values
Resp_surf.columns = [''] + list(vector)
Resp_surf.index = [''] + list(vector)
Resp_surf = Resp_surf.iloc[1:]
Resp_surf = Resp_surf.iloc[:, 1:]
Resp_surf

Ensure that the coefficients array is defined with the appropriate values. This array should contain six elements representing the constant term, the linear coefficients, the interaction coefficient, and the quadratic coefficients.

In [ ]:
Resp_surf_ffact2 = Resp_surf

# Populate the DataFrame
for i in range(11):  # Skip the first row and column
    for j in range(11):
        x = Resp_surf.columns[j]  # Corresponding vector value for the column
        y = Resp_surf.index[i]    # Corresponding vector value for the row
        # Apply the formula
        Resp_surf.iloc[i, j] = (coefficients[0] + coefficients[1]*x + coefficients[2]*y +
                                coefficients[3]*x*y + coefficients[4]*x*x + coefficients[5]*y*y)

# Display the updated DataFrame
#display(Resp_surf)

In [ ]:
Resp_surf

#### Creating a 3D surface plot from the data in your Response Surface matrix

* This type of plot is particularly useful for visualizing the relationship between two factors and a response variable in a clear, intuitive manner.

Interpretation:

* The plot visually represents how changes in 'Factor_1' and 'Factor_2' together affect the yield (response variable). Peaks and valleys in the surface indicate combinations of factor levels that result in high and low yields, respectively.

Optimization:

* This kind of plot can be used to identify optimal combinations of factor levels for maximizing or minimizing the response variable.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

# Assuming Resp_surf2 is your DataFrame

# Create coordinate arrays for the x and y values
X, Y = np.meshgrid(Resp_surf_ffact2.columns.astype(float), Resp_surf_ffact2.index.astype(float))

# Get the z values from the DataFrame and handle NaN or infinite values
Z = Resp_surf_ffact2.values
Z = np.nan_to_num(Z)  # Replace NaN with 0 and infinite with large finite numbers

# Ensure all arrays are of numeric type
X, Y, Z = [np.array(arr, dtype=float) for arr in [X, Y, Z]]

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot a surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add labels and title
ax.set_xlabel(Factor_1_) #Factor_1
ax.set_ylabel(Factor_2_) #Factor_2
ax.set_zlabel('Yield') #Factor_3
ax.set_title('3D Surface Plot')

# Add a color bar
fig.colorbar(surf)

# Save the figure as an HTML file in the 'images' subdirectory
os.makedirs('images', exist_ok=True)
fig.savefig('images/3D_Surface_Plot.png') 

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os

# Assuming Resp_surf_ffact2 is your DataFrame containing the response surface data

# Create coordinate arrays for %B initial and Temperature values
# Adjust the vectors according to the specific layout of your data if needed
X, Y = np.meshgrid(Resp_surf_ffact2.columns.astype(float), Resp_surf_ffact2.index.astype(float))

# Get the z values from the DataFrame and handle NaN or infinite values
Z = Resp_surf_ffact2.values
Z = np.nan_to_num(Z)  # Replace NaN with 0 and infinite with large finite numbers

# Ensure all arrays are of numeric type
X, Y, Z = [np.array(arr, dtype=float) for arr in [X, Y, Z]]

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot a surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Set the correct labels for the axes to reflect the interaction between %B initial and Temperature
ax.set_xlabel(Factor_1_)  # Adjusted to reflect Factor_1
ax.set_ylabel(Factor_3_)  # Adjusted to reflect Factor_3
ax.set_zlabel('Yield')  # The response variable
ax.set_title('3D Surface Plot of %B Initial vs Temperature Interaction')

# Add a color bar for the surface plot
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Save the figure as an image file
os.makedirs('images', exist_ok=True)
fig.savefig('images/3D_Surface_Plot_Initial_Temperature.png')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os

# Assuming Resp_surf_ffact2 is your DataFrame containing the response surface data

# Create coordinate arrays for %B slope and Temperature values
# Adjust the vectors according to the columns and index of your data
X, Y = np.meshgrid(Resp_surf_ffact2.columns.astype(float), Resp_surf_ffact2.index.astype(float))

# Get the z values from the DataFrame and handle NaN or infinite values
Z = Resp_surf_ffact2.values
Z = np.nan_to_num(Z)  # Replace NaN with 0 and infinite with large finite numbers

# Ensure all arrays are of numeric type
X, Y, Z = [np.array(arr, dtype=float) for arr in [X, Y, Z]]

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot a surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Set the correct labels for the axes to reflect the interaction between %B slope and Temperature
ax.set_xlabel('%B slope')  # Adjusted to reflect Factor_2
ax.set_ylabel('Temperature')  # Adjusted to reflect Factor_3
ax.set_zlabel('Yield')  # The response variable
ax.set_title('3D Surface Plot of %B Slope vs Temperature Interaction')

# Add a color bar for the surface plot
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Save the figure as an image file
os.makedirs('images', exist_ok=True)
fig.savefig('images/3D_Surface_Plot_Slope_Temperature.png')

plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Create coordinate arrays for the x and y values
X, Y = np.meshgrid(Resp_surf_ffact2.columns.astype(float), Resp_surf_ffact2.index.astype(float))

# Get the z values from the DataFrame
Z = Resp_surf_ffact2.values

# Create a Plotly 3D surface plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, contours_z=dict(
    show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))])

# Update the layout
fig.update_layout(
    title='3D Surface Plot with Projections',
    scene=dict(
        xaxis_title='%B initial', 
        yaxis_title='%B slope',
        zaxis_title='Intensity',
        aspectmode='manual',  # Adjust the aspect ratio manually
        aspectratio=dict(x=1, y=1, z=0.5)  # Set aspect ratio to make it less slender
    ),
    autosize=True,
    width=1000,
    height=1000,
    margin=dict(l=65, r=50, b=65, t=50)
)

# Save the figure as an HTML file in the 'images' subdirectory
os.makedirs('images', exist_ok=True)
fig.write_html('images/3D_Surface_Plot_interactive.html')

# Show the plot
fig.show()


##### with Temperature

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming design_table includes all factors and interaction terms
# Extracting y-values (dependent variable) and x-values (independent variables)
y = design_table[['Results']].values

# Update X to include Factor_3 and its interaction/squared terms
X = design_table[["Factor_1", "Factor_2", "Factor_3", "Factor_1_2", "Factor_1_1", "Factor_2_2"]].values

# Add a column of ones to X to include an intercept in the model
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Perform linear regression
coefficients, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)

# Display the coefficients
print("Coefficients:", coefficients.flatten())

# Standard error calculation (optional for regression diagnostics)
standard_error = np.sqrt(residuals / (len(y) - len(coefficients)))
print("Standard Error:", standard_error)

# Generate mesh grid for three factors: %B initial, %B slope, and Temperature
vector_1 = np.linspace(min(design_table['Factor_1']), max(design_table['Factor_1']), 11)
vector_2 = np.linspace(min(design_table['Factor_2']), max(design_table['Factor_2']), 11)
vector_3 = np.linspace(min(design_table['Factor_3']), max(design_table['Factor_3']), 11)

# Create mesh grid for three factors
X_grid, Y_grid, Z_grid = np.meshgrid(vector_1, vector_2, vector_3)

# Calculate the response surface based on the regression coefficients
Response = (coefficients[0] +
            coefficients[1] * X_grid +
            coefficients[2] * Y_grid +
            coefficients[3] * Z_grid +
            coefficients[4] * X_grid * Y_grid +
            coefficients[5] * X_grid * X_grid +
            coefficients[6] * Y_grid * Y_grid)

# Create a 3D surface plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Flatten the arrays for plotting
ax.plot_surface(X_grid[:, :, 0], Y_grid[:, :, 0], Response[:, :, 0], cmap='viridis', edgecolor='none')

# Set labels and title
ax.set_xlabel('%B initial')  # Factor_1
ax.set_ylabel('%B slope')    # Factor_2
ax.set_zlabel('Response')    # Response with Temperature (Factor_3)
ax.set_title('3D Surface Plot Including Temperature')

# Add a color bar for reference
fig.colorbar(ax.plot_surface(X_grid[:, :, 0], Y_grid[:, :, 0], Response[:, :, 0], cmap='viridis', edgecolor='none'))

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming design_table includes all factors and interaction terms
# Extracting y-values (dependent variable) and x-values (independent variables)
y = design_table[['Results']].values

# Update X to include Factor_3 (Temperature) and its interaction/squared terms
X = design_table[["Factor_1", "Factor_2", "Factor_3", "Factor_1_2", "Factor_1_1", "Factor_2_2"]].values

# Add a column of ones to X to include an intercept in the model
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Perform linear regression
coefficients, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)

# Display the coefficients
print("Coefficients:", coefficients.flatten())

# Standard error calculation (optional for regression diagnostics)
standard_error = np.sqrt(residuals / (len(y) - len(coefficients)))
print("Standard Error:", standard_error)

# Generate mesh grid for Temperature and %B slope, keeping %B initial at specific levels
vector_2 = np.linspace(min(design_table['Factor_2']), max(design_table['Factor_2']), 11)  # %B slope
vector_3 = np.linspace(min(design_table['Factor_3']), max(design_table['Factor_3']), 11)  # Temperature

# Select levels for %B initial
initial_levels = [min(design_table['Factor_1']), np.mean(design_table['Factor_1']), max(design_table['Factor_1'])]

# Create figure for multiple surface plots
fig = plt.figure(figsize=(18, 6))

# Loop through different levels of %B initial
for i, level in enumerate(initial_levels):
    # Create mesh grid for Temperature vs %B slope
    X_grid, Y_grid = np.meshgrid(vector_2, vector_3)
    
    # Calculate the response surface based on the regression coefficients
    Response = (coefficients[0] +
                coefficients[1] * level +  # %B initial fixed at different levels
                coefficients[2] * X_grid +  # %B slope
                coefficients[3] * Y_grid +  # Temperature
                coefficients[4] * level * X_grid +
                coefficients[5] * level * level +
                coefficients[6] * X_grid * X_grid)

    # Create a subplot for each level of %B initial
    ax = fig.add_subplot(1, 3, i + 1, projection='3d')
    surf = ax.plot_surface(X_grid, Y_grid, Response, cmap='viridis', edgecolor='none')

    # Set plot labels and title
    ax.set_xlabel('%B Slope')
    ax.set_ylabel('Temperature')
    ax.set_zlabel('Response')
    ax.set_title(f'%B initial = {level}')

    # Add a color bar for the response surface
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Sample coefficients for the regression model (replace with actual values)
coefficients = [1, 0.5, 0.2, -0.3, 0.1, 0.05, -0.02]

# Temperature levels to test (e.g., 25°C, 35°C, 45°C)
temperature_levels = [25, 35, 45]

# Vectors for %B Slope and another independent variable (can be adjusted)
vector_2 = np.linspace(0, 10, 50)  # %B Slope
vector_3 = np.linspace(0, 10, 50)  # Another independent variable

# Create a figure for 3D plots
fig = plt.figure(figsize=(18, 6))

# Loop through different levels of Temperature
for i, temp in enumerate(temperature_levels):
    # Create mesh grid for %B Slope vs another independent variable
    X_grid, Y_grid = np.meshgrid(vector_2, vector_3)
    
    # Calculate the response surface based on the regression coefficients
    Response = (coefficients[0] +
                coefficients[1] * X_grid +  # %B Slope
                coefficients[2] * Y_grid +  # Another independent variable
                coefficients[3] * temp +    # Temperature fixed at different levels
                coefficients[4] * X_grid * Y_grid +
                coefficients[5] * temp * temp +
                coefficients[6] * X_grid * X_grid)

    # Create a subplot for each level of Temperature
    ax = fig.add_subplot(1, 3, i + 1, projection='3d')
    surf = ax.plot_surface(X_grid, Y_grid, Response, cmap='viridis', edgecolor='none')

    # Set plot labels and title
    ax.set_xlabel('%B Slope')
    ax.set_ylabel('Another Variable')
    ax.set_zlabel('Response')
    ax.set_title(f'Temperature = {temp}°C')

    # Add a color bar for the response surface
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.tight_layout()
plt.show()
